In [1]:
import torch
import torch.nn.functional as F
from torch import optim

import yaml
import pandas as pd
import numpy as np
import pickle

from sklearn.preprocessing import quantile_transform 
from x_transformers import XTransformer, TransformerWrapper, Decoder, Encoder, ViTransformerWrapper

from aptamer_transformer.model import *
from aptamer_transformer.factories_model_loss import *
from aptamer_transformer.data_utils import *

%load_ext autoreload
%autoreload 2

2024-01-12 15:58:10.090137: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-12 15:58:10.134090: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-12 15:58:10.134130: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-12 15:58:10.135519: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-12 15:58:10.143500: I tensorflow/core/platform/cpu_feature_guar

In [6]:
cfg = read_cfg('../aptamer_transformer/config.yaml')
# Initialize the model and optimizer here as you did during training
device = torch.device("cuda:0")  
cfg.update({
    'device': device,
})

model = get_model(cfg).to(device)

with open('../data/raw_data/nupack_strucutre_data/mfe.pickle', 'rb') as f:
    mfe = pickle.load(f)


In [2]:
cfg = read_cfg('../aptamer_transformer/config.yaml')

df = load_seq_and_struc_data(cfg)

/glade/work/mlsample/conda-envs/guess/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:2663: UserWarning: n_quantiles (1000) is greater than the total number of samples (104). n_quantiles is set to n_samples.
  warnings.warn(
/glade/work/mlsample/conda-envs/guess/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:2663: UserWarning: n_quantiles (1000) is greater than the total number of samples (76). n_quantiles is set to n_samples.
  warnings.warn(
/glade/work/mlsample/conda-envs/guess/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:2663: UserWarning: n_quantiles (1000) is greater than the total number of samples (92). n_quantiles is set to n_samples.
  warnings.warn(
/glade/work/mlsample/conda-envs/guess/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:2663: UserWarning: n_quantiles (1000) is greater than the total number of samples (134). n_quantiles is set to n_samples.
  warnings.warn(
/glade/work/mlsample/conda-envs/guess/lib/python3.10/s

In [ ]:
cfg = read_cfg('../aptamer_transformer/config.yaml')

dna_dataset = get_pytorch_dataset(df, cfg)

In [9]:
save_data_set_as_pickle(dna_dataset, cfg)

In [ ]:
struc_energy = mfe[0][0]

energy = struc_energy.energy
struc = struc_energy.structure

print(f'Dot Bracket Secondart Strucutre Notation:\n{struc.dotparensplus()}\n')
print(f'Adjacency Matrix (edges):\n{struc.matrix()}\n')
print(f'Secondary Strucutre Mean Free Energy:\n{energy}')

In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.processors import TemplateProcessing
from transformers import PreTrainedTokenizerFast
from transformers import DataCollatorForLanguageModeling

tokenizer = Tokenizer(BPE(unk_token="N"), )
tokenizer.pre_tokenizer = Whitespace()
tokenizer.mask_token = "[MASK]"
tokenizer.cls_token = "[CLS]"
tokenizer.pad_token = "[PAD]"
tokenizer.unknown_token = "N"
tokenizer.model_max_length = 42
tokenizer.enable_padding(pad_id=0, pad_token="[PAD]")
tokenizer.post_processor = TemplateProcessing(
    single="[CLS] $A",
    special_tokens=[("[PAD]",0), ("N", 1), ("[CLS]", 2), ("[MASK]", 3)]
)

trainer = BpeTrainer(
    vocab_size=8,
    special_tokens=["[PAD]", "N", "[CLS]", "[MASK]"],
)

structures = [' '.join(df.dot_bracket_struc) for key in mfe.keys()]
tokenizer.train_from_iterator(structures, trainer=trainer)

fast_tokenizer = PreTrainedTokenizerFast(tokenizer_object=tokenizer, model_max_length=42, cls_token="[CLS]", unk_token="N", pad_token="[PAD]", mask_token="[MASK]", return_special_tokens_mask=1)


In [ ]:
tokenized_structures = fast_tokenizer(structures, padding=True, )

In [ ]:
fast_tokenizer.save_pretrained('../data/AptamerBERT_tokenizer')

In [ ]:
from transformers import AutoTokenizer, DataCollatorForLanguageModeling
tokenizer = AutoTokenizer.from_pretrained('../data/AptamerBERT_tokenizer')
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
tokenizer